In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

open_eyes_dir = r'Open_Eyes'  
closed_eyes_dir = r'Closed_Eyes' 
train_open_dir = r'data\train\open'
train_closed_dir = r'data\train\closed'
test_open_dir = r'data\test\open'
test_closed_dir = r'data\test\closed'

# Create directories if they don't exist
os.makedirs(train_open_dir, exist_ok=True)
os.makedirs(train_closed_dir, exist_ok=True)
os.makedirs(test_open_dir, exist_ok=True)
os.makedirs(test_closed_dir, exist_ok=True)

# Function to split and copy images
def split_and_copy_images(source_dir, train_dir, test_dir):
    images = os.listdir(source_dir)
    
    # Split into training and testing sets (80% train, 20% test)
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)
    
    # Copy images to train and test directories
    for image in train_images:
        shutil.copy(os.path.join(source_dir, image), os.path.join(train_dir, image))
    
    for image in test_images:
        shutil.copy(os.path.join(source_dir, image), os.path.join(test_dir, image))

# Split and copy for both categories
split_and_copy_images(open_eyes_dir, train_open_dir, test_open_dir)
split_and_copy_images(closed_eyes_dir, train_closed_dir, test_closed_dir)

print("Data has been split into training and testing sets and saved in the respective folders.")

Data has been split into training and testing sets and saved in the respective folders.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout # type: ignore
from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore
import os

# Paths to the training and validation directories
train_dir = r'data\train'
test_dir = r'data\test'

# Image dimensions and batch size
IMG_HEIGHT = 64
IMG_WIDTH = 64
BATCH_SIZE = 32

# Data augmentation and preprocessing for training and testing
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Load the training and testing data
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'  # Binary classification: open eyes vs. closed eyes
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

# CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
EPOCHS = 10  # You can increase this for better accuracy
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=EPOCHS
)

# Save the model
model.save('eye_status_cnn_model.h5')
print("Model saved as 'eye_status_cnn_model.h5'")


Found 3200 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


c:\Users\vansh\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\vansh\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 53s 498ms/step - accuracy: 0.6696 - loss: 0.5732 - val_accuracy: 0.8612 - val_loss: 0.3039
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 15s 147ms/step - accuracy: 0.8468 - loss: 0.3501 - val_accuracy: 0.9337 - val_loss: 0.2124
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 14s 136ms/step - accuracy: 0.8908 - loss: 0.2668 - val_accuracy: 0.9425 - val_loss: 0.1453
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 23s 229ms/step - accuracy: 0.9080 - loss: 0.2196 - val_accuracy: 0.9513 - val_loss: 0.1253
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 15s 146ms/step - accuracy: 0.9359 - loss: 0.1611 - val_accuracy: 0.9887 - val_loss: 0.0618
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 15s 147ms/step - accuracy: 0.9582 - loss: 0.1252 - val_accuracy: 0.9875 - val_loss: 0.0488
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 14s 139ms/step - accuracy: 0.9685 - loss: 0.0900 - val_accuracy: 0.9987 - val_loss: 0.0203
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 14s 138ms/step - accuracy: 0.9713 - loss: 0

Model saved as 'eye_status_cnn_model.h5'
